As usual, we'll start with cleaning as I won't ever assume data is 'good to go' just because somebody handed it to me.

As a heads up, I find it interesting their focus on RMSLE, something I honstly haven't heard of until a few minutes ago. Likely, a sign that we'll be seeing some logistic relationships, so a heads up on that.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/train.csv/train.csv')
print(df.shape)
df.head() #Quite a nice number of rows...

(90615, 10)


,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [3]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('.', '')
df.columns

Index(['id', 'sex', 'length', 'diameter', 'height', 'whole_weight',
       'whole_weight1', 'whole_weight2', 'shell_weight', 'rings'],
      dtype='object')

In [4]:
# if len(df['id'].unique()) == df.shape[0]:
#     df = df.drop(['id'], axis=1) #Not adding anything to our model... Hmm, now that I think about it, let's make this better:
# else:
#     print(len(df['id'].unique()))

In [5]:
for i in df.columns:
    if len(df[i].unique()) == df.shape[0]:
        df = df.drop([i], axis=1) #Not adding anything to our model... Hmm, now that I think about it, let's make this better:
        print(f"Just dropped {i} as it wasn't adding any value whatsoever to our model.")
    else:
        print(f"Well, {i} has {len(df[i].unique())} unique fields, so we'll keep it...for now!")

#And, on the topic we see all the unique fields too for everything.... Might have to have a different
#approach for larger datasets, but for now... So, pragmatically sex would be the only categorical variable.

Just dropped id as it wasn't adding any value whatsoever to our model.
Well, sex has 3 unique fields, so we'll keep it...for now!
Well, length has 157 unique fields, so we'll keep it...for now!
Well, diameter has 126 unique fields, so we'll keep it...for now!
Well, height has 90 unique fields, so we'll keep it...for now!
Well, whole_weight has 3175 unique fields, so we'll keep it...for now!
Well, whole_weight1 has 1799 unique fields, so we'll keep it...for now!
Well, whole_weight2 has 979 unique fields, so we'll keep it...for now!
Well, shell_weight has 1129 unique fields, so we'll keep it...for now!
Well, rings has 28 unique fields, so we'll keep it...for now!


In [6]:
df.head()

,sex,length,diameter,height,whole_weight,whole_weight1,whole_weight2,shell_weight,rings
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [7]:
#Oh, before I get too excited let's check for nulls:
if sum(df.isnull().sum())==0:
    print("Phew, no nulls.")
else:
    print(df.columns[df.isnull().any()]) #Honestly thanks to leplandelavile of Stackoverflow for the code for this.... I knew
    #we could find something elegant like this,a s opposed to looping through each column...

Phew no nulls.


In [8]:
# df.loc[df.shape[0]] = ["F", 0,0,0,0,0,0,0, np.nan]
# df.loc[df.shape[0]-1]

Given that we only have on categorical vaiable, we'll just look at that plainly:

In [9]:
df['sex'].value_counts(normalize=True) #Oh interesting, I is actually the majority, then a decent bit more males than females.
#Would be interesting to see their sampling methods.

sex
I    0.365204
M    0.342405
F    0.292391
Name: proportion, dtype: float64

Likely visualization would be the way to go, but eh, I'm a numbers guy...

In [28]:
focus  = ['sex']
for i in focus:
    for c in df.columns:
        if c != i:
            print(f"{i} on {c}")
            print(df.groupby(i)[c].describe())
            print(f"~"*26)

sex on length
       count      mean       std    min    25%    50%    75%    max
sex                                                                
F    26495.0  0.584330  0.071348  0.230  0.545  0.585  0.630  0.815
I    33093.0  0.414792  0.104592  0.075  0.345  0.425  0.495  0.770
M    31027.0  0.568806  0.084280  0.090  0.530  0.575  0.620  0.815
~~~~~~~~~~~~~~~~~~~~~~~~~~
sex on diameter
       count      mean       std    min    25%    50%    75%    max
sex                                                                
F    26495.0  0.458219  0.059050  0.130  0.425  0.460  0.500  0.635
I    33093.0  0.315766  0.085249  0.055  0.260  0.325  0.380  0.600
M    31027.0  0.445032  0.069665  0.090  0.415  0.450  0.485  0.650
~~~~~~~~~~~~~~~~~~~~~~~~~~
sex on height
       count      mean       std   min    25%    50%    75%   max
sex                                                              
F    26495.0  0.157194  0.027658  0.05  0.140  0.155  0.175  1.13
I    33093.0  0.103129  